In [ ]:
!pip install --quiet --upgrade "numpy<2"
import importlib, torch
importlib.reload(torch)     # re-imports torch now that NumPy 1.x is present

In [ ]:
# ---- ALWAYS import numpy BEFORE torch / fastai ----
import numpy as np           # NumPy 1.x is now in the env
from fastai.vision.all import load_learner, PILImage
import io, ipywidgets as widgets
from IPython.display import display

# 1) load your FastAI learner
learn = load_learner('model/export.pkl')

# 2) build widgets
uploader = widgets.FileUpload(accept='image/*', multiple=True)
out      = widgets.Output()

def on_upload(change):
    out.clear_output(wait=True)
    with out:
        # handle both tuple (new ipywidgets) and dict (old)
        files = change['new'] if isinstance(change['new'], (tuple, list)) else change['new'].values()
        for info in files:
            name, bts = info['name'], info['content']
            img = PILImage.create(io.BytesIO(bts))          # FastAI handles RGB + dtype
            pred, _, probs = learn.predict(img)
            display(img.to_thumb(256,256))
            print(f"{name} → {pred};  probs = {probs.cpu().numpy().round(4)}\n")


uploader.observe(on_upload, names='value')
display(widgets.VBox([uploader, out]))